In [2]:
## Dependencies for data analysis ##

import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from IPython.display import display
from math import sqrt
%matplotlib inline

# Aggregation

In order to do valuable analysis, we first have to aggregate the data. This is done with the following pieces of code. What we prefer is that we can see, per tag_me, the amount and length of interactions with other tags, with a threshold of 20 seconds. This threshold means that when some tag had an interaction with person 1, and 19 seconds later had another interaction with him, this would be seen as one interaction, instead of two seperate interactions.

In [1]:
import pandas as pd
df = pd.read_csv('/Users/stefan/Google Drive/publicatie/nodeschecked2.csv')
df

,tag_me,tag_them,time_local_s,time_remote_s,rssi,angle,group
0,0x28894773,0x6380E2E2,1335,1284,-74,-3,2
1,0x28894773,0x744C9B55,1336,1907,-72,-3,2
2,0x28894773,0x6380E2E2,1336,1285,-62,-17,2
3,0x28894773,0x744C9B55,1337,1908,-78,0,2
4,0x28894773,0x744C9B55,1338,1909,-67,0,2
5,0x28894773,0x744C9B55,1341,1913,-84,3,2
6,0x28894773,0x77F6F7E8,1454,1565,-81,-56,2
7,0x28894773,0x77F6F7E8,1471,1582,-83,-18,2
8,0x28894773,0x15697F98,1532,1418,-81,-20,2
9,0x28894773,0x345E3277,1535,1844,-78,-5,2


In [2]:
## Testing the functions for aggregating the data clearly
"""
    This aggregation function is now working. This piece of code basically takes care of repeated values in tag_them
    and sums them together. 
"""
df = df.assign(interaction_num=(df.tag_them!=df.tag_them.shift()).cumsum())
gb = (df.groupby('interaction_num').apply(lambda x:pd.Series([x['time_local_s'].min(),x['time_local_s'].max()])).rename(columns={0:'time_start',1:'time_end'}))
df = df.merge(gb,left_on ='interaction_num',right_index=True)
df = (df.assign(total_time = df.time_end - df.time_start).groupby('interaction_num').first().drop('time_local_s',axis=1))
#df.to_csv('output.csv',index=None)
df
#df.total_time.nlargest(10)

,tag_me,tag_them,time_remote_s,rssi,angle,group,time_start,time_end,total_time
interaction_num,,,,,,,,,
1,0x28894773,0x6380E2E2,1284,-74,-3,2,1335,1335,0
2,0x28894773,0x744C9B55,1907,-72,-3,2,1336,1336,0
3,0x28894773,0x6380E2E2,1285,-62,-17,2,1336,1336,0
4,0x28894773,0x744C9B55,1908,-78,0,2,1337,1341,4
5,0x28894773,0x77F6F7E8,1565,-81,-56,2,1454,1471,17
6,0x28894773,0x15697F98,1418,-81,-20,2,1532,1532,0
7,0x28894773,0x345E3277,1844,-78,-5,2,1535,1542,7
8,0x28894773,0x1608F645,1276,-84,-29,2,1549,1549,0
9,0x28894773,0x6979EF0C,1813,-83,-13,2,1576,1576,0


In the following code, we set this threshold of 20 secs and apply it.

In [3]:
## Cleaning and aggregating

"""
    This code is now finally working. What was missing in the original code was that it was also required to check
    whether tag_me shifted == tag_me, because if you don't you get freakishly high values at the points where tag_me 
    changes. Comments are added for clarity.
"""

## Read the CSV
import numpy as np
## Sorting values on tag_them, so we can shift tag_them and see whether they were similar or not
df = df.sort_values(['tag_them'])

## Here we add some copy of original dataset columns, some of whom are shifted.
df = df.assign(TimeValues=lambda df: df.time_start.shift(1))
df = df.assign(OriginalTime=lambda df: df.time_end.shift(0))
df = df.assign(Tag_themShift=lambda df: df.tag_them.shift(0))

## This is where we check whether the next tag_them is equal to the previous tag_them
df = df.assign(Values=lambda df: (df.tag_them==df.tag_them.shift(1)))

## Now we apply that if we have the previous code to be true AND where tag_me == tag_me shifted
## we change the time_end value with the time_end of the previous value. This is done with the time.shift(1). 
df['time_end'] = np.where( ( (df['Values']==True) & (df['tag_me'] == df['tag_me'].shift(1)) ), df['TimeValues'], df['time_end'])

## This is the threshold selector step, if the difference is smaller than 20, we do nothing, if larger
df['time_end'] = np.where( abs((df['time_start'] - df['time_end'] < 20)), df['time_end'], (df['time_start'] + df['total_time']))
df = df.drop(columns=['TimeValues', 'OriginalTime', 'Tag_themShift', 'Values'])

## New total_time values
df['Total_time'] = abs(df['time_end'] - df['time_start'])

## Remove old total_time values
df = df.drop(columns=['total_time'])

## Remove all values where total_time == 0
df = df[(df != 0).all(1)]

## Save to FinalData
#df.to_csv('FinalData.csv', index=None)

## Change indices and sort by time
#df = pd.read_csv('FinalData.csv')
df = df.sort_values('time_start')
df = df.reset_index(drop=True)
#df.to_csv('FinalData.csv', index=None)


In [4]:
df
#df.to_csv('step2.csv', index=None)

,tag_me,tag_them,time_remote_s,rssi,angle,group,time_start,time_end,Total_time
0,0x34FF6AD3,0x744C9B55,1344,-85,4,2,593,5719.0,5126.0
1,0x34FF6AD3,0x6979EF0C,1012,-84,5,2,596,597.0,1.0
2,0x34FF6AD3,0x744C9B55,1350,-82,-11,2,598,5688.0,5090.0
3,0x34FF6AD3,0x3C992634,1052,-85,-4,2,602,5623.0,5021.0
4,0x39E3064D,0x34FF6AD3,352,-81,25,1,604,605.0,1.0
5,0x39E3064D,0x77F6F7E8,650,-83,15,1,612,5574.0,4962.0
6,0x34FF6AD3,0x744C9B55,1384,-83,12,2,633,5696.0,5063.0
7,0x34FF6AD3,0x744C9B55,1384,-83,12,2,633,5644.0,5011.0
8,0x39E3064D,0x77F6F7E8,674,-83,10,1,636,646.0,10.0
9,0x39E3064D,0x3C992634,847,-82,1,1,650,654.0,4.0


Save this data for analysis

In [5]:

criteria2 = df['Total_time'] >= 20
new = df[criteria2]
new
new.Total_time.mean()
new.to_csv('/Users/stefan/Google Drive/publicatie/FinalDataversion2.csv', index=None)

In [6]:
new.Total_time.nlargest(100)

0       5126.0
2       5090.0
6       5063.0
3       5021.0
7       5011.0
113     5000.0
5       4962.0
104     4945.0
14      4747.0
75      4689.0
453     3668.0
384     3581.0
385     3470.0
392     3455.0
554     3445.0
381     3366.0
228     3309.0
244     3266.0
248     3246.0
709     3102.0
714     3098.0
232     3074.0
225     3060.0
770     3018.0
768     3010.0
769     3008.0
773     3001.0
560     2978.0
794     2955.0
443     2939.0
         ...  
966     2516.0
815     2507.0
862     2497.0
570     2493.0
643     2491.0
746     2485.0
798     2442.0
688     2436.0
204     2410.0
816     2395.0
430     2394.0
944     2386.0
1069    2365.0
1167    2357.0
852     2336.0
857     2331.0
1097    2313.0
630     2299.0
904     2270.0
1227    2231.0
432     2200.0
997     2179.0
821     2144.0
343     2124.0
855     2122.0
1057    2102.0
1018    2086.0
865     2084.0
1062    2083.0
469     2067.0
Name: Total_time, Length: 100, dtype: float64